# リッジ回帰を行う

## 準備

In [1]:
import pandas as pd
import numpy as np
import glob
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Ridge
from sklearn import preprocessing

In [2]:
stock_name_list = glob.glob("../output/*.csv")
stock_name_list

['../output\\LSIML_RV_grid_data_33820_1.csv',
 '../output\\LSIML_RV_grid_data_40630_1.csv',
 '../output\\LSIML_RV_grid_data_44520_1.csv',
 '../output\\LSIML_RV_grid_data_45020_1.csv',
 '../output\\LSIML_RV_grid_data_45030_1.csv',
 '../output\\LSIML_RV_grid_data_45680_1.csv',
 '../output\\LSIML_RV_grid_data_60980_1.csv',
 '../output\\LSIML_RV_grid_data_62730_1.csv',
 '../output\\LSIML_RV_grid_data_63670_1.csv',
 '../output\\LSIML_RV_grid_data_65010_1.csv',
 '../output\\LSIML_RV_grid_data_65940_1.csv',
 '../output\\LSIML_RV_grid_data_67580_1.csv',
 '../output\\LSIML_RV_grid_data_68610_1.csv',
 '../output\\LSIML_RV_grid_data_69540_1.csv',
 '../output\\LSIML_RV_grid_data_69810_1.csv',
 '../output\\LSIML_RV_grid_data_72030_1.csv',
 '../output\\LSIML_RV_grid_data_72670_1.csv',
 '../output\\LSIML_RV_grid_data_77410_1.csv',
 '../output\\LSIML_RV_grid_data_79740_1.csv',
 '../output\\LSIML_RV_grid_data_80010_1.csv',
 '../output\\LSIML_RV_grid_data_80310_1.csv',
 '../output\\LSIML_RV_grid_data_80

In [3]:
#df = pd.read_csv(stock_name_list[-1])
# df = pd.read_csv('../output\\LSIML_RV_grid_data_83160_1.csv',)
df = pd.read_csv('../output\\LSIML_RV_grid_data_79740_1.csv',)
df

,date,RV_sec1,LSIML_b=10,LSIML_b=50,LSIML_b=100,Num_jump_b=10,Num_jump_b=50,Num_jump_b=100,Size_jump_b=10,Size_jump_b=50,Size_jump_b=100
0,2018-09-03,0.000304,0.000126,0.000129,0.000124,1,3,8,0.000060,0.000056,0.000074
1,2018-09-04,0.000291,0.000088,0.000093,0.000087,1,4,9,0.000084,0.000068,0.000066
2,2018-09-05,0.000286,0.000116,0.000103,0.000111,1,6,9,0.000110,0.000099,0.000093
3,2018-09-06,0.000574,0.000210,0.000187,0.000210,1,6,8,0.000230,0.000259,0.000193
4,2018-09-07,0.000364,0.000124,0.000156,0.000134,1,2,8,0.000092,0.000052,0.000097
...,...,...,...,...,...,...,...,...,...,...,...
699,2021-07-26,0.000185,0.000070,0.000066,0.000069,1,6,11,0.000090,0.000082,0.000096
700,2021-07-27,0.000102,0.000032,0.000048,0.000042,2,5,10,0.000033,0.000039,0.000036
701,2021-07-28,0.000154,0.000053,0.000066,0.000069,1,5,8,0.000039,0.000054,0.000050
702,2021-07-29,0.000161,0.000061,0.000071,0.000059,1,4,12,0.000081,0.000068,0.000081


## データを読み込み、加工する

In [4]:
sec = 50
LSIML = df["LSIML_b=" + str(sec)] + df["Size_jump_b=" + str(sec)]
LSIML

0      0.000185
1      0.000161
2      0.000202
3      0.000446
4      0.000209
         ...   
699    0.000148
700    0.000088
701    0.000120
702    0.000140
703    0.000143
Length: 704, dtype: float64

### 移動平均, 一日ずれデータを計算する

In [5]:
week = LSIML.rolling(5).mean().shift()
month = LSIML.rolling(22).mean().shift()

In [6]:
month.dropna()

22     0.000210
23     0.000213
24     0.000214
25     0.000215
26     0.000211
         ...   
699    0.000146
700    0.000148
701    0.000138
702    0.000138
703    0.000139
Length: 682, dtype: float64

In [7]:
shifted_LSIML = LSIML.shift()
shifted_LSIML

0           NaN
1      0.000185
2      0.000161
3      0.000202
4      0.000446
         ...   
699    0.000171
700    0.000148
701    0.000088
702    0.000120
703    0.000140
Length: 704, dtype: float64

In [8]:
data = pd.concat([LSIML, shifted_LSIML, week, month], axis=1)
data.columns = ["tom", "tod", "week", "month"]
data

,tom,tod,week,month
0,0.000185,NaN,NaN,NaN
1,0.000161,0.000185,NaN,NaN
2,0.000202,0.000161,NaN,NaN
3,0.000446,0.000202,NaN,NaN
4,0.000209,0.000446,NaN,NaN
...,...,...,...,...
699,0.000148,0.000171,0.000166,0.000146
700,0.000088,0.000148,0.000169,0.000148
701,0.000120,0.000088,0.000169,0.000138
702,0.000140,0.000120,0.000153,0.000138


### ジャンプサイズ

In [9]:
data["jump"] = df["Size_jump_b=" + str(sec)].shift()

### 対数を取る

In [10]:
data["log_tom"] = np.log(data["tom"])
data["log_tod"] = np.log(data["tod"])
data["log_week"] = np.log(data["week"])
data["log_month"] = np.log(data["month"])
data

,tom,tod,week,month,jump,log_tom,log_tod,log_week,log_month
0,0.000185,NaN,NaN,NaN,NaN,-8.597496,NaN,NaN,NaN
1,0.000161,0.000185,NaN,NaN,0.000056,-8.731708,-8.597496,NaN,NaN
2,0.000202,0.000161,NaN,NaN,0.000068,-8.505452,-8.731708,NaN,NaN
3,0.000446,0.000202,NaN,NaN,0.000099,-7.715394,-8.505452,NaN,NaN
4,0.000209,0.000446,NaN,NaN,0.000259,-8.474643,-7.715394,NaN,NaN
...,...,...,...,...,...,...,...,...,...
699,0.000148,0.000171,0.000166,0.000146,0.000109,-8.817087,-8.673788,-8.705866,-8.829185
700,0.000088,0.000148,0.000169,0.000148,0.000082,-9.343210,-8.817087,-8.683346,-8.816924
701,0.000120,0.000088,0.000169,0.000138,0.000039,-9.028031,-9.343210,-8.684307,-8.889513
702,0.000140,0.000120,0.000153,0.000138,0.000054,-8.877237,-9.028031,-8.783488,-8.889932


In [11]:
data["log_jump"] = np.log(1 + preprocessing.scale(data["jump"]))
data

,tom,tod,week,month,jump,log_tom,log_tod,log_week,log_month,log_jump
0,0.000185,NaN,NaN,NaN,NaN,-8.597496,NaN,NaN,NaN,NaN
1,0.000161,0.000185,NaN,NaN,0.000056,-8.731708,-8.597496,NaN,NaN,-0.622438
2,0.000202,0.000161,NaN,NaN,0.000068,-8.505452,-8.731708,NaN,NaN,-0.445048
3,0.000446,0.000202,NaN,NaN,0.000099,-7.715394,-8.505452,NaN,NaN,-0.108553
4,0.000209,0.000446,NaN,NaN,0.000259,-8.474643,-7.715394,NaN,NaN,0.801445
...,...,...,...,...,...,...,...,...,...,...
699,0.000148,0.000171,0.000166,0.000146,0.000109,-8.817087,-8.673788,-8.705866,-8.829185,-0.017780
700,0.000088,0.000148,0.000169,0.000148,0.000082,-9.343210,-8.817087,-8.683346,-8.816924,-0.281822
701,0.000120,0.000088,0.000169,0.000138,0.000039,-9.028031,-9.343210,-8.684307,-8.889513,-0.913886
702,0.000140,0.000120,0.000153,0.000138,0.000054,-8.877237,-9.028031,-8.783488,-8.889932,-0.645769


### 日次データを読み込む

In [12]:
# daily_data = pd.read_csv("../data/daily_data/Daily_Price_83160_1.csv").shift()
daily_data = pd.read_csv("../data/daily_data/Daily_Price_79740_1.csv").shift()
daily_data

,date,log_price,daily_return
0,NaN,NaN,NaN
1,2018-09-03,10.598882,NaN
2,2018-09-04,10.590365,-0.008517
3,2018-09-05,10.587846,-0.002519
4,2018-09-06,10.551114,-0.036732
...,...,...,...
699,2021-07-21,10.999095,-0.002504
700,2021-07-26,11.004098,0.005003
701,2021-07-27,11.009241,0.005143
702,2021-07-28,10.988339,-0.020902


In [13]:
asym_list = []
for i in range(len(daily_data)):
    asym_list.append(min([0, daily_data["daily_return"].iloc[i]]))

In [14]:
data["asym"] = asym_list

### 欠損値を削除する

In [15]:
data = data.dropna()
data

,tom,tod,week,month,jump,log_tom,log_tod,log_week,log_month,log_jump,asym
22,0.000253,0.000328,0.000178,0.000210,0.000134,-8.284074,-8.023400,-8.635981,-8.468394,0.171637,-0.013152
23,0.000191,0.000253,0.000199,0.000213,0.000123,-8.561726,-8.284074,-8.524631,-8.453796,0.090929,0.000000
24,0.000217,0.000191,0.000209,0.000214,0.000030,-8.436525,-8.561726,-8.472662,-8.447438,-1.114013,-0.025579
25,0.000356,0.000217,0.000227,0.000215,0.000077,-7.940341,-8.436525,-8.391433,-8.444382,-0.339672,-0.005997
26,0.000314,0.000356,0.000269,0.000211,0.000132,-8.066678,-7.940341,-8.221255,-8.463545,0.157557,-0.031830
...,...,...,...,...,...,...,...,...,...,...,...
699,0.000148,0.000171,0.000166,0.000146,0.000109,-8.817087,-8.673788,-8.705866,-8.829185,-0.017780,-0.002504
700,0.000088,0.000148,0.000169,0.000148,0.000082,-9.343210,-8.817087,-8.683346,-8.816924,-0.281822,0.000000
701,0.000120,0.000088,0.000169,0.000138,0.000039,-9.028031,-9.343210,-8.684307,-8.889513,-0.913886,0.000000
702,0.000140,0.000120,0.000153,0.000138,0.000054,-8.877237,-9.028031,-8.783488,-8.889932,-0.645769,-0.020902


# リッジ回帰で予測する

In [16]:
data

,tom,tod,week,month,jump,log_tom,log_tod,log_week,log_month,log_jump,asym
22,0.000253,0.000328,0.000178,0.000210,0.000134,-8.284074,-8.023400,-8.635981,-8.468394,0.171637,-0.013152
23,0.000191,0.000253,0.000199,0.000213,0.000123,-8.561726,-8.284074,-8.524631,-8.453796,0.090929,0.000000
24,0.000217,0.000191,0.000209,0.000214,0.000030,-8.436525,-8.561726,-8.472662,-8.447438,-1.114013,-0.025579
25,0.000356,0.000217,0.000227,0.000215,0.000077,-7.940341,-8.436525,-8.391433,-8.444382,-0.339672,-0.005997
26,0.000314,0.000356,0.000269,0.000211,0.000132,-8.066678,-7.940341,-8.221255,-8.463545,0.157557,-0.031830
...,...,...,...,...,...,...,...,...,...,...,...
699,0.000148,0.000171,0.000166,0.000146,0.000109,-8.817087,-8.673788,-8.705866,-8.829185,-0.017780,-0.002504
700,0.000088,0.000148,0.000169,0.000148,0.000082,-9.343210,-8.817087,-8.683346,-8.816924,-0.281822,0.000000
701,0.000120,0.000088,0.000169,0.000138,0.000039,-9.028031,-9.343210,-8.684307,-8.889513,-0.913886,0.000000
702,0.000140,0.000120,0.000153,0.000138,0.000054,-8.877237,-9.028031,-8.783488,-8.889932,-0.645769,-0.020902


元系列に対して、非対称性が存在するか確かめる

In [17]:
x_names = ["tod", "week", "month", "asym", "jump"]
ridge = Ridge().fit(data[x_names], data["tom"])
# 係数を出力する
res_coef = pd.DataFrame(ridge.coef_).T 
res_coef.columns = x_names
res_coef

,tod,week,month,asym,jump
0,0.000027,0.000022,0.00001,-0.000474,0.000012


## 予測を行う

In [18]:
def ridge_rolling(X, Y, not_log_y = False, log_flag=False, gauussian_flag = False):
    """
    リッジ回帰でローリングウィンドウの一期先予測を行う
    ハイパーパラメータαは[0.001, 0.011, 0.021,..., 0.091]からクロスバリデーションで選択する
    """
    # 学習データを作成
    train_length = len(X) // 2

    MSE_list = []
    MAE_list = []
    HMSE_list = []
    for i in range(len(X) - train_length):
        # 学習データを作成
        train_x = X.iloc[i : train_length + i]
        train_y = Y.iloc[i : train_length + i]

        # テストデータを作成
        test_x = X.iloc[train_length + i]
        # 変形
        test_x = np.array(test_x).reshape([1, len(test_x)])
        
        # モデルを作成
        mod1 = RidgeCV(alphas=np.array(list(range(1, 101, 20))) * 0.000001, cv=10)
        mod1.fit(train_x, train_y)
        
        # 残差を計算する
        residual = mod1.predict(train_x) - train_y
        # 残差分散
        var_res = np.var(residual)
        #print(var_res)

        #print(mod1.alpha_)

        # 予測を行う
        pred_y = mod1.predict(test_x)

        if log_flag:
            # 対数モデルの場合、対数を取っていない真値を別に与える
            test_y = not_log_y.iloc[train_length + i]

            if gauussian_flag:
                # 誤差項の分布に正規分布を仮定する
                pred_y = np.exp(pred_y + var_res / 2)
            else:
                # 誤差項の分布を特に仮定しない（単にexp変換を行う）
                pred_y = np.exp(pred_y)
        else:
            test_y = Y.iloc[train_length + i]


        dif_ = (test_y - pred_y) ** 2
        abs_ = abs(test_y - pred_y)
        hmse_ = (1 - pred_y / test_y) ** 2

        MSE_list.append(float(dif_))
        MAE_list.append(float(abs_))
        HMSE_list.append(float(hmse_))

    MSE = np.mean(MSE_list)
    MAE = np.mean(MAE_list)
    HMSE = np.mean(HMSE_list)

    return {
        "MSE": MSE,
        "MAE": MAE,
        "HMSE": HMSE,
    }

### 非対数モデル

In [19]:
result_list = []

In [20]:
x_names = ["tod", "week", "month"]
X = data[x_names]
Y = data["tom"]

result = ridge_rolling(X, Y)

result_list.append(result)
result

{'MSE': 3.8945160721772524e-08,
 'MAE': 8.732849689258253e-05,
 'HMSE': 0.25335502586650316}

In [21]:
x_names = ["tod", "week", "month", "jump"]
X = data[x_names]
Y = data["tom"]

result = ridge_rolling(X, Y)
result_list.append(result)
result

{'MSE': 3.979516557162621e-08,
 'MAE': 8.731532927009434e-05,
 'HMSE': 0.2508551380262098}

In [22]:
x_names = ["tod", "week", "month", "asym"]
X = data[x_names]
Y = data["tom"]

result = ridge_rolling(X, Y)
result_list.append(result)
result

{'MSE': 3.847719521317993e-08,
 'MAE': 8.849470976086359e-05,
 'HMSE': 0.25467670360631833}

In [23]:
x_names = ["tod", "week", "month", "asym", "jump"]
X = data[x_names]
Y = data["tom"]

result = ridge_rolling(X, Y)
result_list.append(result)
result

{'MSE': 3.926872350472584e-08,
 'MAE': 8.84799611443413e-05,
 'HMSE': 0.25216615509905255}

### 対数モデル（誤差項に正規分布を仮定しない）

In [24]:
x_names = ["log_tod", "log_week", "log_month"]
X = data[x_names]
Y = data["log_tom"]

result = ridge_rolling(X, Y, data["tom"], log_flag=True, gauussian_flag = False)
result_list.append(result)
result

{'MSE': 3.761006185510471e-08,
 'MAE': 8.146634885323918e-05,
 'HMSE': 0.17091104315901454}

In [25]:
x_names = ["log_tod", "log_week", "log_month", "log_jump"]
X = data[x_names]
Y = data["log_tom"]

result = ridge_rolling(X, Y, data["tom"], log_flag=True, gauussian_flag = False)
result_list.append(result)
result

{'MSE': 3.6449989439222184e-08,
 'MAE': 7.971178441173905e-05,
 'HMSE': 0.15602715346312546}

In [26]:
x_names = ["log_tod", "log_week", "log_month", "asym"]
X = data[x_names]
Y = data["log_tom"]

result = ridge_rolling(X, Y, data["tom"], log_flag=True, gauussian_flag = False)
result_list.append(result)
result

{'MSE': 3.6635463693881816e-08,
 'MAE': 7.981667000291763e-05,
 'HMSE': 0.16208698660472753}

In [27]:
x_names = ["log_tod", "log_week", "log_month", "log_jump", "asym"]
X = data[x_names]
Y = data["log_tom"]

result = ridge_rolling(X, Y, data["tom"], log_flag=True, gauussian_flag = False)
result_list.append(result)
result

{'MSE': 3.568710903431191e-08,
 'MAE': 7.866424670878923e-05,
 'HMSE': 0.149337515196144}

### 対数モデル（誤差項に正規分布を仮定）

In [28]:
x_names = ["log_tod", "log_week", "log_month"]
X = data[x_names]
Y = data["log_tom"]

result = ridge_rolling(X, Y, data["tom"], log_flag=True, gauussian_flag = True)
result_list.append(result)
result

{'MSE': 3.700038709507089e-08,
 'MAE': 8.598255765343633e-05,
 'HMSE': 0.22885846033413795}

In [29]:
x_names = ["log_tod", "log_week", "log_month", "log_jump"]
X = data[x_names]
Y = data["log_tom"]

result = ridge_rolling(X, Y, data["tom"], log_flag=True, gauussian_flag = True)
result_list.append(result)
result

{'MSE': 3.549671728245058e-08,
 'MAE': 8.356658344776564e-05,
 'HMSE': 0.20686096209381857}

In [30]:
x_names = ["log_tod", "log_week", "log_month", "asym"]
X = data[x_names]
Y = data["log_tom"]

result = ridge_rolling(X, Y, data["tom"], log_flag=True, gauussian_flag = True)
result_list.append(result)
result

{'MSE': 3.5741313622609225e-08,
 'MAE': 8.338414041601466e-05,
 'HMSE': 0.21515930021297336}

In [31]:
x_names = ["log_tod", "log_week", "log_month", "log_jump", "asym"]
X = data[x_names]
Y = data["log_tom"]

result = ridge_rolling(X, Y, data["tom"], log_flag=True, gauussian_flag = True)
result_list.append(result)
result

{'MSE': 3.452245615391527e-08,
 'MAE': 8.152961699710155e-05,
 'HMSE': 0.1964154127147192}

In [32]:
# 桁を設定
pd.options.display.precision = 4

MSE = []
MAE = []
HMSE = []

for res in result_list:
    MSE.append(res["MSE"])
    MAE.append(res["MAE"])
    HMSE.append(res["HMSE"])

result_df = pd.DataFrame([MSE, MAE, HMSE]).T
result_df.columns = ["MSE", "MAE", "HMSE"]
result_df

,MSE,MAE,HMSE
0,3.8945e-08,8.7328e-05,0.2534
1,3.9795e-08,8.7315e-05,0.2509
2,3.8477e-08,8.8495e-05,0.2547
3,3.9269e-08,8.8480e-05,0.2522
4,3.7610e-08,8.1466e-05,0.1709
5,3.6450e-08,7.9712e-05,0.1560
6,3.6635e-08,7.9817e-05,0.1621
7,3.5687e-08,7.8664e-05,0.1493
8,3.7000e-08,8.5983e-05,0.2289
9,3.5497e-08,8.3567e-05,0.2069


In [33]:
# indexを変更
result_df.index = ["HAR", "HAR-J", "HAR-X", "HAR-JX", "HAR(log)", "HAR-J(log)", "HAR-X(log)", "HAR-JX(log)", "HAR(log)*", "HAR-J(log)*", "HAR-X(log)*", "HAR-JX(log)*"]

In [34]:
#latex用に出力
print(result_df.to_latex())

\begin{tabular}{lrrr}
\toprule
{} &         MSE &         MAE &    HMSE \\
\midrule
HAR          &  3.8945e-08 &  8.7328e-05 &  0.2534 \\
HAR-J        &  3.9795e-08 &  8.7315e-05 &  0.2509 \\
HAR-X        &  3.8477e-08 &  8.8495e-05 &  0.2547 \\
HAR-JX       &  3.9269e-08 &  8.8480e-05 &  0.2522 \\
HAR(log)     &  3.7610e-08 &  8.1466e-05 &  0.1709 \\
HAR-J(log)   &  3.6450e-08 &  7.9712e-05 &  0.1560 \\
HAR-X(log)   &  3.6635e-08 &  7.9817e-05 &  0.1621 \\
HAR-JX(log)  &  3.5687e-08 &  7.8664e-05 &  0.1493 \\
HAR(log)*    &  3.7000e-08 &  8.5983e-05 &  0.2289 \\
HAR-J(log)*  &  3.5497e-08 &  8.3567e-05 &  0.2069 \\
HAR-X(log)*  &  3.5741e-08 &  8.3384e-05 &  0.2152 \\
HAR-JX(log)* &  3.4522e-08 &  8.1530e-05 &  0.1964 \\
\bottomrule
\end{tabular}



In [35]:
# result_df.to_csv("result_" + "79740_"+ str(sec) + ".csv")